### Content Based Recommendation System

In [65]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Read the Dataset `movies_metadata.csv`

In [0]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [67]:
movie=pd.read_csv('gdrive/My Drive/AIML/movies_metadata.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [68]:
movie.dtypes

adult                    object 
belongs_to_collection    object 
budget                   object 
genres                   object 
homepage                 object 
id                       object 
imdb_id                  object 
original_language        object 
original_title           object 
overview                 object 
popularity               object 
poster_path              object 
production_companies     object 
production_countries     object 
release_date             object 
revenue                  float64
runtime                  float64
spoken_languages         object 
status                   object 
tagline                  object 
title                    object 
video                    object 
vote_average             float64
vote_count               float64
dtype: object

In [69]:
movie.shape

(45466, 24)

In [70]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
adult                    45466 non-null object
belongs_to_collection    4494 non-null object
budget                   45466 non-null object
genres                   45466 non-null object
homepage                 7782 non-null object
id                       45466 non-null object
imdb_id                  45449 non-null object
original_language        45455 non-null object
original_title           45466 non-null object
overview                 44512 non-null object
popularity               45461 non-null object
poster_path              45080 non-null object
production_companies     45463 non-null object
production_countries     45463 non-null object
release_date             45379 non-null object
revenue                  45460 non-null float64
runtime                  45203 non-null float64
spoken_languages         45460 non-null object
status                   45379 non-null objec

In [71]:
movie.isna().sum()

adult                    0    
belongs_to_collection    40972
budget                   0    
genres                   0    
homepage                 37684
id                       0    
imdb_id                  17   
original_language        11   
original_title           0    
overview                 954  
popularity               5    
poster_path              386  
production_companies     3    
production_countries     3    
release_date             87   
revenue                  6    
runtime                  263  
spoken_languages         6    
status                   87   
tagline                  25054
title                    6    
video                    6    
vote_average             6    
vote_count               6    
dtype: int64

In [72]:
movie.head(100)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",NaN,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'ba

### Create a new column with name 'description' combining `'overview' and 'tagline'` columns in the given dataset

In [73]:
movie['tagline'].fillna('')

0                                                                                      
1        Roll the dice and unleash the excitement!                                     
2        Still Yelling. Still Fighting. Still Ready for Love.                          
3        Friends are the people who let you be yourself... and never let you forget it.
4        Just When His World Is Back To Normal... He's In For The Surprise Of His Life!
                                              ...                                      
45461    Rising and falling between a man and woman                                    
45462                                                                                  
45463    A deadly game of wits.                                                        
45464                                                                                  
45465                                                                                  
Name: tagline, Length: 45466, dt

In [74]:
movie['overview'].fillna('')

0        Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.                                                                                                                                                                                                                                                                                                                                                                                                         
1        When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves ri

In [0]:
movie['description']=movie['tagline']+movie['overview']

In [76]:
movie.shape

(45466, 25)

### Lets drop the null values in `description` column

In [77]:
movie.isna().sum()

adult                    0    
belongs_to_collection    40972
budget                   0    
genres                   0    
homepage                 37684
id                       0    
imdb_id                  17   
original_language        11   
original_title           0    
overview                 954  
popularity               5    
poster_path              386  
production_companies     3    
production_countries     3    
release_date             87   
revenue                  6    
runtime                  263  
spoken_languages         6    
status                   87   
tagline                  25054
title                    6    
video                    6    
vote_average             6    
vote_count               6    
description              25062
dtype: int64

In [0]:
movie.dropna(subset=['description'], inplace=True)


In [79]:
movie.isna().sum()

adult                    0    
belongs_to_collection    17832
budget                   0    
genres                   0    
homepage                 15994
id                       0    
imdb_id                  4    
original_language        0    
original_title           0    
overview                 0    
popularity               0    
poster_path              15   
production_companies     0    
production_countries     0    
release_date             14   
revenue                  0    
runtime                  0    
spoken_languages         0    
status                   14   
tagline                  0    
title                    0    
video                    0    
vote_average             0    
vote_count               0    
description              0    
dtype: int64

### Keep the first occurance and drop duplicates of each title in column `title`

In [0]:
movie['title'].drop_duplicates(inplace=True)

In [81]:
movie.shape

(20404, 25)

In [82]:
movie['title'].head(10)

1     Jumanji                    
2     Grumpier Old Men           
3     Waiting to Exhale          
4     Father of the Bride Part II
5     Heat                       
6     Sabrina                    
7     Tom and Huck               
8     Sudden Death               
9     GoldenEye                  
10    The American President     
Name: title, dtype: object

### As we might have dropped a few rows with duplicate `title` in above step, just reset the index [make sure you are not adding any new column to the dataframe while doing reset index]

In [83]:
movie.reset_index()

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,description
0,1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"Roll the dice and unleash the excitement!When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures."
1,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",NaN,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0,"Still Yelling. Still Fighting. Still Ready for Love.A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max."
2,3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34.0,"Friends are the people who let you be yourself... and never let you forget it.Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elus

In [84]:
movie.shape

(20404, 25)

In [85]:
movie

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,description
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"Roll the dice and unleash the excitement!When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",NaN,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0,"Still Yelling. Still Fighting. Still Ready for Love.A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34.0,"Friends are the people who let you be yourself... and never let you forget it.Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good m

### Generate tf-idf matrix using the column `description`. Consider till 3-grams, with minimum document frequency as 0.

In [0]:
tf_idf_vect = TfidfVectorizer(analyzer='word',ngram_range=(1,3),stop_words='english', min_df = 0)
tf_idf_vect.fit(movie['description'])
title_matrix = tf_idf_vect.transform(movie["description"])

In [87]:
title_matrix

<20404x1202356 sparse matrix of type '<class 'numpy.float64'>'
	with 1905258 stored elements in Compressed Sparse Row format>

In [88]:
print(title_matrix)

  (0, 1192603)	0.10368949805420052
  (0, 1192600)	0.09440248709871014
  (0, 1191346)	0.03695673324891967
  (0, 1183850)	0.10368949805420052
  (0, 1183849)	0.10368949805420052
  (0, 1180344)	0.0314268611519634
  (0, 1120172)	0.10368949805420052
  (0, 1120171)	0.09957993023900759
  (0, 1120123)	0.06722915068598863
  (0, 1116678)	0.10368949805420052
  (0, 1116677)	0.10368949805420052
  (0, 1116652)	0.0712695343326798
  (0, 1088910)	0.10368949805420052
  (0, 1088901)	0.0808716621093798
  (0, 1088719)	0.05546617019508469
  (0, 1056556)	0.10368949805420052
  (0, 1056503)	0.05618721664064755
  (0, 960312)	0.10368949805420052
  (0, 960311)	0.10368949805420052
  (0, 960283)	0.07190988038345225
  (0, 910839)	0.10368949805420052
  (0, 910838)	0.10368949805420052
  (0, 910714)	0.058195187470428046
  (0, 905169)	0.10368949805420052
  (0, 905168)	0.10368949805420052
  :	:
  (20403, 497556)	0.18247919097440876
  (20403, 497555)	0.18247919097440876
  (20403, 497503)	0.11132777649826106
  (20403, 44652

In [89]:
title_matrix.shape

(20404, 1202356)

### create cosine similarity matrix

In [0]:
cosine_sim = cosine_similarity(title_matrix)

In [91]:
cosine_sim.shape

(20404, 20404)

In [0]:
movie=movie.reset_index()

In [0]:
indices = pd.Series(movie.index, index=movie['title'])

### Write a function with name `recommend` which takes `title` as argument and returns a list of 10 recommended title names in the output based on the above cosine similarities

In [0]:
def recommend(title_id,cosine_sim):
    indexx=indices[title_id]
    top_n_idx = np.flip(np.argsort(cosine_sim[indexx,]),axis=0)[0:11]
    top_n_sim_values = cosine_sim[indexx, top_n_idx]
    return movie['title'].iloc[top_n_idx]

### Give the recommendations from above functions for movies `The Godfather` and `The Dark Knight Rises`

In [95]:
recommend('The Godfather',cosine_sim)

611      The Godfather         
865      The Godfather: Part II
16565    Honor Thy Father      
12436    The Family            
13078    Blood Ties            
3256     Made                  
4047     Johnny Dangerously    
19387    Live by Night         
6008     Fury                  
26       Shanghai Triad        
19841    In Memory of My Father
Name: title, dtype: object

In [96]:
recommend('The Dark Knight Rises',cosine_sim)

10957    The Dark Knight Rises             
8307     The Dark Knight                   
112      Batman Forever                    
988      Batman Returns                    
2352     Batman: Mask of the Phantasm      
451      Batman                            
12323    Batman: Mystery of the Batwoman   
9772     Batman: Under the Red Hood        
6502     Batman Beyond: Return of the Joker
14018    Batman vs Dracula                 
10859    Batman: Year One                  
Name: title, dtype: object